# Not Used. For Testing Purposes Only.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import pandas as pd 

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score

In [3]:
#Load csv
df = pd.read_csv("./dataset/tbpred.csv")

In [136]:
#Label Encode 'Country of Birth' 
le = preprocessing.LabelEncoder()
df['encoded_country'] = le.fit_transform(df['CountryofBirth'])

In [137]:
X = df.drop(['Name','CountryofBirth'], axis=1)

In [138]:
def metrics_calculator(y_test, y_pred, model_name):
    '''
    This function calculates all desired performance metrics for a given model.
    '''
    result = pd.DataFrame(data=[accuracy_score(y_test, y_pred),
                                precision_score(y_test, y_pred, average='macro'),
                                recall_score(y_test, y_pred, average='macro'),
                                f1_score(y_test, y_pred, average='macro')],
                          index=['Accuracy','Precision','Recall','F1-score'],
                          columns = [model_name])
    return result

In [139]:
X2 = X.drop('tb_pred', axis=1)
Y2 = X['tb_pred'].values.reshape(-1, 1)

In [140]:
# Check the shape of X and y
print ('X:', X2.shape,'\nY:', Y2.shape)

X: (2000, 22) 
Y: (2000, 1)


In [141]:
# Split X and y into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split (X2, Y2, stratify=Y2, test_size = 0.25, random_state = 0)

In [142]:
# Instantiate the model
DT_model = DecisionTreeClassifier(max_depth=3)

# Fit the model to the training set
DT_model.fit(X_train2, y_train2)

DecisionTreeClassifier(max_depth=3)

In [143]:
y_pred = DT_model.predict(X_test2)

In [144]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test2, y_pred)))

Model accuracy score: 0.7640


In [145]:
print(classification_report(y_test2, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86       380
           1       0.52      0.24      0.33       120

    accuracy                           0.76       500
   macro avg       0.66      0.59      0.59       500
weighted avg       0.73      0.76      0.73       500



In [146]:
BaseDT_result = metrics_calculator(y_test2, y_pred, 'Base Decision Tree')
BaseDT_result

,Base Decision Tree
Accuracy,0.764000
Precision,0.656451
Recall,0.585307
F1-score,0.593171


In [147]:
#Finding optimal hyperparameters(GridSearchCV)

# Define model
model = DecisionTreeClassifier()

# Define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)

# Define search parameters
max_depth = range(2, 19, 1)
criterion = ['entropy', 'gini']   

#min_samples_split = [2, 3, 4]
#min_samples_leaf = [1, 2, 3]

param_grid = {'max_depth': max_depth, 'criterion':criterion}

# Define search
search = GridSearchCV(model, param_grid, scoring='accuracy', n_jobs=1, cv=cv)

# Execute search
GridSearchCV = search.fit(X_train2, y_train2)

# Set the clf to the best combination of parameters
DT_modelcv = GridSearchCV.best_estimator_

# Summarize result
print('Best Score: %s' % GridSearchCV.best_score_)
print('Best Hyperparameters: %s' % GridSearchCV.best_params_)

Best Score: 0.8175555555555554
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 5}


In [148]:
DT_modelcv.fit(X_train2, y_train2)

DecisionTreeClassifier(max_depth=5)

In [149]:
y_pred2 = DT_modelcv.predict(X_test2)

In [150]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test2, y_pred2)))

Model accuracy score: 0.7920


In [151]:
print(classification_report(y_test2, y_pred2))

              precision    recall  f1-score   support

           0       0.83      0.91      0.87       380
           1       0.60      0.41      0.49       120

    accuracy                           0.79       500
   macro avg       0.71      0.66      0.68       500
weighted avg       0.77      0.79      0.78       500



In [152]:
TunedDT_result = metrics_calculator(y_test2, y_pred2, 'Tuned Decision Tree')
TunedDT_result

,Tuned Decision Tree
Accuracy,0.792000
Precision,0.713852
Recall,0.660746
F1-score,0.677411


In [153]:
# To save an encoder
from joblib import dump
dump(le, 'label_encoder.joblib')

# To load the encoder
from joblib import load
le = load('label_encoder.joblib')

from joblib import dump
dump(DT_modelcv, 'DT_modelcv.joblib')

['DT_modelcv.joblib']